## Data Reading (CEI DATA) - CG

In [1]:
import pandas as pd
import os

csvs_dir='../data/submission2/cg_comm_counters'

col_names=['version','app','class','cpu','threads','metric','value']

lst_csv_dfs=[]

for csv_filename in os.listdir(csvs_dir):
    if os.path.isdir(csvs_dir+'/'+csv_filename)==True:
        continue
    lst_csv_dfs.append(pd.read_csv(csvs_dir+'/'+csv_filename, names=col_names))

df_all_cei=pd.concat(lst_csv_dfs)
df_all_cei=df_all_cei.sort_values(['version','app','class','threads','cpu'])

#drop some rows with multiple header names
df_all_cei=df_all_cei.loc[df_all_cei['value']!='value'].reset_index(drop=True)
#casting to numeric
#df_all_cei.loc[:,'version']=pd.to_numeric(df_all_cei['version'])
df_all_cei.loc[:,'cpu']=pd.to_numeric(df_all_cei['cpu'])
df_all_cei.loc[:,'threads']=pd.to_numeric(df_all_cei['threads'])
df_all_cei.loc[:,'value']=pd.to_numeric(df_all_cei['value'])
df_all_cei


,version,app,class,cpu,threads,metric,value
0,native-L1,cg,A,0,1,MEM-LOAD-L3-HIT-RETIRED-XSNP-HIT,0
1,native-L1,cg,A,0,1,MEM-LOAD-L3-HIT-RETIRED-XSNP-MISS,0
2,native-L1,cg,A,0,1,MEM-LOAD-L3-HIT-RETIRED-XSNP-NONE,840115
3,native-L1,cg,A,0,1,MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM,0
4,native-L1,cg,A,0,1,MEM-LOAD-L3-HIT-RETIRED-XSNP-NONE,853011
...,...,...,...,...,...,...,...
61114,native-none,cg,W,7,8,MEM-LOAD-L3-HIT-RETIRED-XSNP-MISS,572
61115,native-none,cg,W,7,8,MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM,573471
61116,native-none,cg,W,7,8,MEM-LOAD-L3-HIT-RETIRED-XSNP-MISS,81
61117,native-none,cg,W,7,8,MEM-LOAD-L3-HIT-RETIRED-XSNP-NONE,2706804


## Data Preprocessing (CEI DATA)

In [3]:
######## Getting the List of counters ###########################
lst_metrics=df_all_cei.groupby(by='metric').count().reset_index()['metric'].to_list()
lst_metrics
#################################################################

['MEM-LOAD-L3-HIT-RETIRED-XSNP-HIT',
 'MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM',
 'MEM-LOAD-L3-HIT-RETIRED-XSNP-MISS',
 'MEM-LOAD-L3-HIT-RETIRED-XSNP-NONE']

## L3 DATA

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import Javascript,display

#hw_counters=['offcore-response-pf-l2-rfo-l3-hit-any-snoop']
metrics=lst_metrics
nb_threads=['all','1','2','4','8','12']
classes=['W','A','B']

#df_miss_non0=df_miss.loc[df_miss['cpu']==0]
def plot_metrics_graph(metric='MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM',nb_threads='all',_class='A'):

    #apps=['bt','cg','ep','ft','is','lu','mg','sp','ua']
    apps=['cg']
    
    ##filtering for nas class and metric
    ##the value of the metric is in the column 'value'
    df_metric=df_all_cei.loc[df_all_cei['class']==_class]
    df_metric=df_metric.loc[df_metric['metric']==metric]
    
    if nb_threads!='all':
        df_metric=df_metric.loc[df_metric['threads']==int(nb_threads)]
    #print(df_metric)

    #sns.catplot(x='threads',y='value',hue='version',data=cycles_max_per_threads,kind='bar',col='app',col_wrap=3)
    sns.set(font_scale=1.5)
    #fig, axs=plt.subplots(nrows=3,ncols=3,figsize=(15,15))
    plt.tight_layout(pad=3.5)

    for i,app in enumerate(apps):
        #row=int(i/3)
        #col=int(i%3)
        cycles_app=df_metric.loc[df_metric['app']==app]
        #print(app)        
        if nb_threads=='all':
            ax=sns.barplot(x='threads',y='value',hue='version',data=cycles_app,
                        hue_order=['native-L1-L2','native-L2','native-L1','native-none'],palette='cubehelix')
            # legend title
            ax.legend(framealpha=0.4)
            new_title = 'Execution'
            ax.legend_.set_title(new_title)            
            # replace labels
            new_labels = ['Real-L1-L2', 'Real-L2', 'Real-L1', 'Real-NoPref']
            for t, l in zip(ax.legend_.texts, new_labels): t.set_text(l)
            #sns.barplot(x='threads',y=metric,hue='version',data=cycles_app,ax=axs[row][col], hue_order=['native-L1-L2','native-L2','native-L1','native-none','zsim','sniper-none','sniper-l1','sniper-l1-l2'])
        else:
            sns.barplot(x='cpu',y='value',hue='version',data=cycles_app,hue_order=['native-L1-L2','native-L2','native-L1','native-none'],palette='cubehelix')
            ax.legend_.remove()
        
        #sns.barplot(x='threads',y='value',hue='version',data=cycles_app,ax=axs[row][col])
        #sns.catplot(x='threads',y='value',hue='version',data=cycles_app,kind='bar',ax=axs[row][col])
        ax.set_title(app.upper()+' (class '+_class+')')
        ax.set_ylabel(metric)
        if nb_threads=='all':
            ax.set_xlabel('Number of Threads')
        else:
            ax.set_xlabel('Core Index')
        #axs[row][col].set_yscale('log')
        #break

    plt.savefig('../figures/submission2/CG_'+metric+'_'+_class+'_'+nb_threads+'_subm2_cei.pdf',type='pdf',bbox_inches='tight')
    
dropdown_metrics=widgets.Dropdown(
    options=metrics,
    value='MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM',
    description='metric',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)   
dropdown_classes=widgets.Dropdown(
    options=classes,
    value='A',
    description='class',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
dropdown_nb_threads=widgets.Dropdown(
    options=nb_threads,
    value='all',
    description='nb_threads',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

interact_manual(plot_metrics_graph,                
                metric=dropdown_metrics, 
                nb_threads=dropdown_nb_threads, 
                _class=dropdown_classes)

interactive(children=(Dropdown(description='metric', index=1, options=('MEM-LOAD-L3-HIT-RETIRED-XSNP-HIT', 'ME…

<function __main__.plot_metrics_graph(metric='MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM', nb_threads='all', _class='A')>

## Reading standard data to plot mem access plot

In [7]:
import pandas as pd
import os

csvs_dir='../data/submission2/native/nas/cei'

col_names=['version','app','class','cpu','threads','metric','value']
lst_metrics=['PAPI-TOT-INS', 
             'PAPI-TOT-CYC',             
             'L2-RQSTS-DEMAND-DATA-RD-HIT', 
             'L2-RQSTS-DEMAND-DATA-RD-MISS', 
             'L2-RQSTS-ALL-PF', 
             'L2-RQSTS-PF-HIT', 
             'L2-RQSTS-PF-MISS', 
             'L2-LINES-OUT-USELESS-HWPREF', 
             'L2-LINES-OUT-USELESS-HWPF', 
             'OFFCORE-RESPONSE-0-PF-L2-DATA-RD-ANY-RESPONSE', 
             'OFFCORE-RESPONSE-0-PF-L2-DATA-RD-L3-HIT-SNP-ANY', 
             'OFFCORE-RESPONSE-0-PF-L2-DATA-RD-L3-HIT-SNP-HITM', 
             'OFFCORE-RESPONSE-0-PF-L2-RFO-ANY-RESPONSE', 
             'OFFCORE-RESPONSE-0-PF-L2-RFO-L3-HIT-SNP-ANY', 
             'OFFCORE-RESPONSE-0-PF-L2-RFO-L3-HIT-SNP-HITM', 
             'OFFCORE-REQUESTS-DEMAND-DATA-RD', 
             'OFFCORE-REQUESTS-ALL-DATA-RD', 
             'OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD']

lst_csv_dfs=[]

for csv_filename in os.listdir(csvs_dir):
    if os.path.isdir(csvs_dir+'/'+csv_filename)==True:
        continue
    lst_csv_dfs.append(pd.read_csv(csvs_dir+'/'+csv_filename, names=col_names))

df_all=pd.concat(lst_csv_dfs)
df_all=df_all.sort_values(['version','app','class','threads','cpu'])
                          
df_all

,version,app,class,cpu,threads,metric,value
333,native-L1,bt,A,0,1,OFFCORE-RESPONSE-0-PF-L2-RFO-L3-HIT-SNP-HITM,0
1186,native-L1,bt,A,0,1,OFFCORE-RESPONSE-0-PF-L2-RFO-ANY-RESPONSE,0
2059,native-L1,bt,A,0,1,OFFCORE-REQUESTS-ALL-DATA-RD,926020907
2073,native-L1,bt,A,0,1,L2-RQSTS-PF-HIT,2611081437
3543,native-L1,bt,A,0,1,OFFCORE-REQUESTS-DEMAND-DATA-RD,131740440
...,...,...,...,...,...,...,...
10726,native-none,ua,W,11,12,L2-RQSTS-ALL-PF,295645
10946,native-none,ua,W,11,12,OFFCORE-REQUESTS-ALL-DATA-RD,1291787
11241,native-none,ua,W,11,12,OFFCORE-RESPONSE-0-PF-L2-DATA-RD-ANY-RESPONSE,0
12838,native-none,ua,W,11,12,PAPI-TOT-CYC,1598817862


In [10]:
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import Javascript,display

#hw_counters=['offcore-response-pf-l2-rfo-l3-hit-any-snoop']
metrics=lst_metrics
nb_threads=['all','1','2','4','8','12']
classes=['W','A','B']

#df_miss_non0=df_miss.loc[df_miss['cpu']==0]
def plot_metrics_graph(metric='MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM',nb_threads='all',_class='A'):

    #apps=['bt','cg','ep','ft','is','lu','mg','sp','ua']
    apps=['cg']
    
    ##filtering for nas class and metric
    ##the value of the metric is in the column 'value'
    df_metric=df_all.loc[df_all['class']==_class]
    df_metric=df_metric.loc[df_metric['metric']==metric]
    
    if nb_threads!='all':
        df_metric=df_metric.loc[df_metric['threads']==int(nb_threads)]
    #print(df_metric)

    #sns.catplot(x='threads',y='value',hue='version',data=cycles_max_per_threads,kind='bar',col='app',col_wrap=3)
    sns.set(font_scale=1.5)
    #fig, axs=plt.subplots(nrows=3,ncols=3,figsize=(15,15))
    plt.tight_layout(pad=3.5)

    for i,app in enumerate(apps):
        #row=int(i/3)
        #col=int(i%3)
        cycles_app=df_metric.loc[df_metric['app']==app]
        #print(app)        
        if nb_threads=='all':
            ax=sns.barplot(x='threads',y='value',hue='version',data=cycles_app,
                        hue_order=['native-L1-L2','native-L2','native-L1','native-none'],palette='cubehelix')
            # legend title
            ax.legend(framealpha=0.4)
            new_title = 'Execution'
            ax.legend_.set_title(new_title)            
            # replace labels
            new_labels = ['Real-L1-L2', 'Real-L2', 'Real-L1', 'Real-NoPref']
            for t, l in zip(ax.legend_.texts, new_labels): t.set_text(l)
            #sns.barplot(x='threads',y=metric,hue='version',data=cycles_app,ax=axs[row][col], hue_order=['native-L1-L2','native-L2','native-L1','native-none','zsim','sniper-none','sniper-l1','sniper-l1-l2'])
        else:
            sns.barplot(x='cpu',y='value',hue='version',data=cycles_app,hue_order=['native-L1-L2','native-L2','native-L1','native-none'],palette='cubehelix')
            ax.legend_.remove()
        
        #sns.barplot(x='threads',y='value',hue='version',data=cycles_app,ax=axs[row][col])
        #sns.catplot(x='threads',y='value',hue='version',data=cycles_app,kind='bar',ax=axs[row][col])
        ax.set_title(app.upper()+' (class '+_class+')')
        ax.set_ylabel(metric)
        if nb_threads=='all':
            ax.set_xlabel('Number of Threads')
        else:
            ax.set_xlabel('Core Index')
        ax.set_yscale('log')
        #break

    plt.savefig('../figures/submission2/CG_'+metric+'_'+_class+'_'+nb_threads+'_subm2_cei.pdf',type='pdf',bbox_inches='tight')
    
dropdown_metrics=widgets.Dropdown(
    options=metrics,
    value='OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD',
    description='metric',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)   
dropdown_classes=widgets.Dropdown(
    options=classes,
    value='A',
    description='class',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
dropdown_nb_threads=widgets.Dropdown(
    options=nb_threads,
    value='all',
    description='nb_threads',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

interact_manual(plot_metrics_graph,                
                metric=dropdown_metrics, 
                nb_threads=dropdown_nb_threads, 
                _class=dropdown_classes)

interactive(children=(Dropdown(description='metric', index=17, options=('PAPI-TOT-INS', 'PAPI-TOT-CYC', 'L2-RQ…

<function __main__.plot_metrics_graph(metric='MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM', nb_threads='all', _class='A')>